In [1]:
import numpy as np
import random

# Parameters Config

In [2]:
SUB      = 0
SUB_ADD  = 1
MULTIPLY = 2

GEN_TYPE = SUB

TRAINING_SIZE = 80000
DIGITS = 3
ANS_DIGITS = {
    SUB: DIGITS + 1,
    SUB_ADD: DIGITS + 1,
    MULTIPLY: 2 * DIGITS
}.get(GEN_TYPE, DIGITS + 1)
MAXLEN = DIGITS + 1 + DIGITS
chars = {
    SUB: '0123456789- ',
    SUB_ADD: '0123456789+- ',
    MULTIPLY: '0123456789* '
}.get(GEN_TYPE, '0123456789+-* ')

# Data Generation

In [3]:
questions = []
expected = []
seen = set()
operator = {
    SUB: ['-'],
    SUB_ADD: ['-', '+'],
    MULTIPLY: ['*']
}
ans_switcher = {
    '+': lambda a, b: a + b,
    '-': lambda a, b: a - b,
    '*': lambda a, b: a * b
}
ops = operator.get(GEN_TYPE, [None])
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: random.choice(range(10 ** random.choice(range(1, DIGITS + 1))))
    g = lambda: random.choice(ops)
    a, b, op = f(), f(), g()
    if op == '-':
        a, b = sorted((a, b), reverse=True)
    key = tuple((a, b, op))
    if key in seen:
        continue
    seen.add(key)
    query = '{}{}{}'.format(a, op, b).ljust(MAXLEN)
    ans_funct = ans_switcher.get(op, lambda a, b: float('NAN'))
    ans = str(ans_funct(a, b)).ljust(ANS_DIGITS)
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [4]:
for i in range(10):
    print(questions[i], '=', expected[i])

875-9   = 866 
93-9    = 84  
77-5    = 72  
82-79   = 3   
876-249 = 627 
67-1    = 66  
84-75   = 9   
400-2   = 398 
559-337 = 222 
346-8   = 338 


# Processing
- Size of training data:   64,000
- Size of validation data: 16,000

In [5]:
class CharacterTable:
    def __init__(self, chars):
        self.chars  = list(chars)
        self.len    = len(chars)
        self.encode = {}
        for i, key in enumerate(self.chars):
            self.encode[key] = np.zeros(self.len, np.float32)
            self.encode[key][i] = 1.
            
    def encoder(self, C):
        result = np.zeros((len(C), self.len))
        for i, c in enumerate(C):
            try:
                result[i] = self.encode[c]
            except:
                pass
        return result
            
    def decoder(self, x):
        x = x.argmax(axis=-1)
        return ''.join(self.chars[i] for i in x)

In [6]:
ct = CharacterTable(chars)

x = np.zeros((len(questions), MAXLEN, len(chars)), np.float32)
y = np.zeros((len(expected), ANS_DIGITS, len(chars)), np.float32)
for i, sentence in enumerate(questions):
    x[i] = ct.encoder(sentence)
for i, sentence in enumerate(expected):
    y[i] = ct.encoder(sentence)

train_x = x[:18000]
train_y = y[:18000]

validation_x = x[18000:20000]
validation_y = y[18000:20000]

# Model

![model](img/seq2seq.png)

- Using sequence to sequence model
- Encoder: bi-directional LSTM
- Decoder: LSTM

In [7]:
import keras as K
from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation, Lambda
from keras.layers import Input, LSTM, TimeDistributed, Dropout, RepeatVector, Reshape, Bidirectional, Concatenate, Embedding
from keras.layers.normalization import BatchNormalization
from keras.models import load_model

HIDDEN_SIZE = 256
EMBEDDING_SIZE = 256

model = Sequential()

encoder_inputs = Input(shape=(MAXLEN, len(chars)))
# encoder_inputs = Input(shape=(MAXLEN,))
# embedding = Embedding(len(chars), EMBEDDING_SIZE)(encoder_inputs)
encoder = Bidirectional(LSTM(HIDDEN_SIZE, return_state=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
states = [state_h, state_c]

# decoder_inputs = RepeatVector(ANS_DIGITS)(encoder_outputs)
# decoder_outputs = Bidirectional(LSTM(int(HIDDEN_SIZE / 2), return_sequences=True))(decoder_inputs)
# decoder_outputs = TimeDistributed(Dense(len(chars), activation='softmax'))(decoder_outputs)

# Set up the decoder, which will only process one timestep at a time.
decoder_inputs = Reshape((1, HIDDEN_SIZE * 2))
decoder_lstm = LSTM(HIDDEN_SIZE * 2, return_state=True)

all_outputs = []
inputs = decoder_inputs(encoder_outputs)

first_decoder = True
for _ in range(ANS_DIGITS):
    # Run the decoder on one timestep
    outputs, state_h, state_c = decoder_lstm(inputs,
                                             initial_state=states)
    
    # Reinject the outputs as inputs for the next loop iteration
    # as well as update the states
    states = [state_h, state_c]
#     inputs = decoder_inputs(outputs)
    
    # Store the current prediction (we will concatenate all predictions later)
    outputs = Dense(len(chars), activation='softmax')(outputs)
    all_outputs.append(outputs)
    

# Concatenate all predictions
decoder_outputs = Concatenate()(all_outputs)
decoder_outputs = Reshape((ANS_DIGITS, len(chars)))(decoder_outputs)
decoder_outputs = Lambda(lambda x: x[:,::-1])(decoder_outputs)

# Define and compile model as previously
model = Model(encoder_inputs, decoder_outputs)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

batch_size = int(len(train_x) / 128 / 100) * 100

if batch_size == 0:
    batch_size = 100

    
model.fit(train_x, train_y, 
          batch_size=batch_size, epochs=100, 
          verbose=1, validation_data=[validation_x, validation_y])

# model.fit(np.argmax(train_x, axis=2), train_y, 
#           batch_size=batch_size, epochs=100, 
#           verbose=1, validation_data=[np.argmax(validation_x, axis=2), validation_y])

/home/ddnn_user02/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 7, 12)        0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) [(None, 512), (None, 550912      input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1, 512)       0           bidirectional_1[0][0]            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 512)          0           bidirectional_1[0][1]            
                                                                 bidirectional_1[0][3]            
__________

18000/18000 [==============================] - 9s 501us/step - loss: 0.0023 - acc: 1.0000 - val_loss: 0.0069 - val_acc: 0.9990
Epoch 27/100
18000/18000 [==============================] - 9s 498us/step - loss: 0.0812 - acc: 0.9732 - val_loss: 0.0872 - val_acc: 0.9708
Epoch 28/100
18000/18000 [==============================] - 9s 512us/step - loss: 0.0305 - acc: 0.9913 - val_loss: 0.0314 - val_acc: 0.9896
Epoch 29/100
18000/18000 [==============================] - 9s 507us/step - loss: 0.0164 - acc: 0.9955 - val_loss: 0.0221 - val_acc: 0.9938
Epoch 30/100
18000/18000 [==============================] - 9s 497us/step - loss: 0.0213 - acc: 0.9934 - val_loss: 0.0150 - val_acc: 0.9959
Epoch 31/100
18000/18000 [==============================] - 9s 507us/step - loss: 0.0144 - acc: 0.9957 - val_loss: 0.0737 - val_acc: 0.9743
Epoch 32/100
18000/18000 [==============================] - 9s 497us/step - loss: 0.0482 - acc: 0.9848 - val_loss: 0.0125 - val_acc: 0.9971
Epoch 33/100
18000/18000 [=======

18000/18000 [==============================] - 9s 495us/step - loss: 6.4462e-06 - acc: 1.0000 - val_loss: 0.0018 - val_acc: 0.9999
Epoch 84/100
18000/18000 [==============================] - 9s 501us/step - loss: 5.8839e-06 - acc: 1.0000 - val_loss: 0.0017 - val_acc: 0.9999
Epoch 85/100
18000/18000 [==============================] - 9s 497us/step - loss: 5.3657e-06 - acc: 1.0000 - val_loss: 0.0017 - val_acc: 0.9999
Epoch 86/100
18000/18000 [==============================] - 9s 482us/step - loss: 4.8978e-06 - acc: 1.0000 - val_loss: 0.0017 - val_acc: 0.9999
Epoch 87/100
18000/18000 [==============================] - 9s 508us/step - loss: 4.4676e-06 - acc: 1.0000 - val_loss: 0.0018 - val_acc: 0.9999
Epoch 88/100
18000/18000 [==============================] - 9s 490us/step - loss: 4.0811e-06 - acc: 1.0000 - val_loss: 0.0018 - val_acc: 0.9999
Epoch 89/100
18000/18000 [==============================] - 9s 483us/step - loss: 3.7238e-06 - acc: 1.0000 - val_loss: 0.0018 - val_acc: 0.9999
Epoch

# Error check

In [8]:
print('Error check')
pred = model.predict(x)
err = []
for i in range(len(x)):
    if ct.decoder(pred[i]) != ct.decoder(y[i]):
        err.append(i)
print(len(err), '/', len(x))
print()
print('Prediction'.ljust(MAXLEN + ANS_DIGITS + 3),'Ans')
print('-' * (MAXLEN + ANS_DIGITS * 2 + 4))
for i in err:
    print(ct.decoder(x[i]), '=', ct.decoder(pred[i]), ct.decoder(y[i]))


Error check
103 / 80000

Prediction     Ans
-------------------
100-98  = 1    2   
290-269 = 22   21  
280-259 = 22   21  
191-189 = 1    2   
207-199 = 7    8   
398-330 = 69   68  
930-888 = 43   42  
660-609 = 52   51  
204-196 = 7    8   
699-510 = 199  189 
895-808 = 77   87  
869-860 = 8    9   
507-499 = 7    8   
690-529 = 151  161 
100-97  = 1    3   
103-97  = 1    6   
199-138 = 71   61  
500-496 = 3    4   
100-92  = 9    8   
100-39  = 71   61  
104-98  = 1    6   
592-502 = 80   90  
901-893 = 7    8   
720-688 = 22   32  
900-818 = 92   82  
260-259 = 2    1   
900-895 = 6    5   
102-94  = 9    8   
721-679 = 43   42  
900-298 = 601  602 
104-99  = 1    5   
102-98  = 1    4   
990-948 = 43   42  
930-878 = 53   52  
491-428 = 64   63  
800-695 = 10   105 
902-888 = 1    14  
290-227 = 64   63  
596-488 = 1    108 
689-500 = 199  189 
670-639 = 32   31  
960-298 = 661  662 
101-97  = 14   4   
104-96  = 9    8   
794-789 = 4    5   
172-109 = 62   63  
870-789 = 82   8

# Test

In [9]:
q = '100-5'

q_padding = q.ljust(MAXLEN)[:MAXLEN]
test_x = ct.encoder(q_padding)
pred_y = model.predict(test_x.reshape(-1, MAXLEN, len(chars)))
print(q, '=', ct.decoder(pred_y[0]))


100-5 = 95  
